## Which state produces the best three-point shoooters?



## Step 1

### We'll start by looking at the three-point accuracy for all the players in each state.


In [0]:
from pandas.io import gbq
project_id = '[YOUR_PROJECT_ID]'

In [4]:
states_3pt_pct_q = """
SELECT 
  birthplace_state AS state,
  SUM(three_points_made) AS threes_made,
  SUM(three_points_att) AS threes_att,
  SUM(three_points_made)/SUM(three_points_att) AS three_point_pct
FROM `bigquery-public-data.ncaa_basketball.mbb_players_games_sr`
WHERE (three_points_att > 0)
AND division_alias = "D1"
AND birthplace_state IN (
    'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI',
    'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MT', 'NE', 'NV', 'NH',
    'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'MD', 'MA', 'MI', 'MN',
    'MS', 'MO', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA',
    'WV', 'WI', 'WY')
GROUP BY state
ORDER BY three_point_pct DESC
"""

states_3pt_pct = gbq.read_gbq(query=states_3pt_pct_q, dialect ='standard', project_id=project_id)
states_3pt_pct.head(10)

Requesting query... ok.
Job ID: job_Hkuezxe0tPYX9TFDkit7616nWPqf
Query running...
Query done.
Processed: 15.9 MB
Standard price: $0.00 USD

Retrieving results...
Got 51 rows.

Total time taken 1.97 s.
Finished at 2018-03-05 15:44:04.


,state,threes_made,threes_att,three_point_pct
0,ND,282,735,0.383673
1,ID,768,2034,0.377581
2,OR,2918,7748,0.376613
3,WY,319,848,0.376179
4,CO,3731,10055,0.371059
5,IA,3280,8928,0.367384
6,NE,1376,3753,0.366640
7,WI,6087,16690,0.364709
8,AZ,4792,13150,0.364411
9,UT,2628,7213,0.364342


## Step 2

### North Dakota comes out on top. But let's dig into what that number means. Let's look at how many players come from each state, and compare them to the overall state population.

In [5]:
num_players_q = """
SELECT
  stats.state as state_abbrev,
  pops.state,
  COUNT(player_id) AS num_players,
  pops.population_2017 AS num_people,
  COUNT(player_id) / pops.population_2017 AS players_per_person
FROM (
  SELECT 
    player_id,
    birthplace_state AS state,
    SUM(three_points_att) AS threes_att
  FROM `bigquery-public-data.ncaa_basketball.mbb_players_games_sr`
  WHERE division_alias = "D1"
  AND birthplace_state IN (
      'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI',
      'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MT', 'NE', 'NV', 'NH',
      'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'MD', 'MA', 'MI', 'MN',
      'MS', 'MO', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA',
      'WV', 'WI', 'WY')
  GROUP BY player_id, birthplace_state
) AS stats
LEFT JOIN `ncaa-basketball-extras.extras.state_population` as pops on pops.abbreviation = stats.state
GROUP BY stats.state, pops.state, pops.population_2017
ORDER BY players_per_person ASC
"""

num_players = gbq.read_gbq(query=num_players_q, dialect ='standard', project_id=project_id)
num_players.head(10)

Requesting query... ok.
Job ID: job_QUnJoAHRF3I2BqyrhW_ru3exHh_M
Query running...
Query done.
Processed: 38.1 MB
Standard price: $0.00 USD

Retrieving results...
Got 51 rows.

Total time taken 2.13 s.
Finished at 2018-03-05 15:53:11.


,state_abbrev,state,num_players,num_people,players_per_person
0,HI,Hawaii,12,1427538,0.000008
1,ME,Maine,15,1335907,0.000011
2,ND,North Dakota,10,755393,0.000013
3,NM,New Mexico,32,2088070,0.000015
4,VT,Vermont,10,623657,0.000016
5,NH,New Hampshire,23,1342795,0.000017
6,RI,Rhode Island,19,1059639,0.000018
7,MA,Massachusetts,131,6859819,0.000019
8,SD,South Dakota,17,869666,0.000020
9,AZ,Arizona,150,7016270,0.000021


## Step 3

### Looks like Hawaii, Maine, and North Dakota have very few players per capita. Getting back to North Dakota though, how many of those 10 players have attempted a three-point shot?

In [6]:
query = """
SELECT
  stats.state as state_abbrev,
  pops.state,
  COUNT(player_id) AS num_players,
  pops.population_2017 AS num_people,
  COUNT(player_id) / pops.population_2017 AS players_per_person
FROM (
  SELECT 
    player_id,
    birthplace_state AS state,
    SUM(three_points_att) AS threes_att
  FROM `bigquery-public-data.ncaa_basketball.mbb_players_games_sr`
  WHERE division_alias = "D1"
  AND birthplace_state IN (
      'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI',
      'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MT', 'NE', 'NV', 'NH',
      'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'MD', 'MA', 'MI', 'MN',
      'MS', 'MO', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA',
      'WV', 'WI', 'WY')
  GROUP BY player_id, birthplace_state
) AS stats
LEFT JOIN `ncaa-basketball-extras.extras.state_population` as pops on pops.abbreviation = stats.state
WHERE threes_att > 0
GROUP BY stats.state, pops.state, pops.population_2017
ORDER BY players_per_person ASC
"""

states = gbq.read_gbq(query=query, dialect ='standard', project_id=project_id)
states.head(10)

Requesting query... ok.
Job ID: job_d1kauXjxLclCVvFSdmwJ1PXbwxG5
Query running...
Query done.
Processed: 42.8 MB
Standard price: $0.00 USD

Retrieving results...
Got 51 rows.

Total time taken 2.07 s.
Finished at 2018-03-05 15:53:46.


,state_abbrev,state,num_players,num_people,players_per_person
0,ND,North Dakota,4,755393,0.000005
1,HI,Hawaii,8,1427538,0.000006
2,ME,Maine,10,1335907,0.000007
3,VT,Vermont,5,623657,0.000008
4,NM,New Mexico,23,2088070,0.000011
5,MT,Montana,14,1050493,0.000013
6,WY,Wyoming,8,579315,0.000014
7,MA,Massachusetts,96,6859819,0.000014
8,NH,New Hampshire,20,1342795,0.000015
9,CO,Colorado,84,5607154,0.000015


## Step 4

### Looks like four players have attempted three-pointers. Out of curiosity, out of those four, how many have made any three-pointers?

In [4]:
query = """
SELECT
  stats.state as state_abbrev,
  pops.state,
  COUNT(player_id) AS num_players,
  pops.population_2017 AS num_people,
  COUNT(player_id) / pops.population_2017 AS players_per_person
FROM (
  SELECT 
    player_id,
    birthplace_state AS state,
    SUM(three_points_made) AS threes_made
  FROM `bigquery-public-data.ncaa_basketball.mbb_players_games_sr`
  WHERE division_alias = "D1"
  AND birthplace_state IN (
      'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI',
      'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MT', 'NE', 'NV', 'NH',
      'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'MD', 'MA', 'MI', 'MN',
      'MS', 'MO', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA',
      'WV', 'WI', 'WY')
  GROUP BY player_id, birthplace_state
) AS stats
LEFT JOIN `ncaa-basketball-extras.extras.state_population` as pops on pops.abbreviation = stats.state
WHERE threes_made > 0
GROUP BY stats.state, pops.state, pops.population_2017
ORDER BY players_per_person ASC
"""

states = gbq.read_gbq(query=query, dialect ='standard', project_id=project_id)
states.head(10)

Requesting query... ok.
Job ID: job_pRkf8JSTdHKFlbSvqWA3bIXKwszr
Query running...
Query done.
Processed: 42.8 MB
Standard price: $0.00 USD

Retrieving results...
Got 51 rows.

Total time taken 2.07 s.
Finished at 2018-03-05 19:33:04.


,state_abbrev,state,num_players,num_people,players_per_person
0,ND,North Dakota,3,755393,0.000004
1,HI,Hawaii,6,1427538,0.000004
2,ME,Maine,8,1335907,0.000006
3,VT,Vermont,4,623657,0.000006
4,NM,New Mexico,19,2088070,0.000009
5,MT,Montana,12,1050493,0.000011
6,WY,Wyoming,7,579315,0.000012
7,MA,Massachusetts,84,6859819,0.000012
8,SD,South Dakota,11,869666,0.000013
9,CO,Colorado,74,5607154,0.000013


## Only three out of the 10 players from North Dakota have made a three-pointer! North Dakota produces great three-point shooters, but it produces very, very few of them.